In [0]:
import json
import transformers
from transformers import BertTokenizer
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
# import keras
import nltk
from nltk.tokenize.util import align_tokens
nltk.download('punkt')
# from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
import torch as pt

In [0]:
with open("/dbfs/FileStore/Chanchal/Data_json/new.json", "r") as f:
  train_data = json.load(f)


In [0]:
with open("//dbfs/FileStore/Chanchal/Datasets/cdr.json", "r") as f:
  test_data = json.load(f)


In [0]:
string_labels = []
for item in train_data:
  text = item['Text']
  id= item['id']
  sentences_split = nltk.word_tokenize(item["Text"])
  # print(sentences_split)
  try:
      token_spans = align_tokens(sentences_split,text)
  except:
      print("Error")
      # print(token_spans)
  for i in range(len(token_spans)):
      token_in_annotation = False
      for annotation in item['annotation']:
          if int(annotation["start"])<= token_spans[i][0] and int(annotation['end'])>=token_spans[i][1]:
              string_labels.append((text[token_spans[i][0]:token_spans[i][1]],annotation['type']))
              token_in_annotation = True
      if(token_in_annotation==False):
          string_labels.append((text[token_spans[i][0]:token_spans[i][1]],'O'))



Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


In [0]:
#addding data to dataframe
train_df = pd.DataFrame(string_labels, columns =['token', 'label'])

In [0]:
# Frequency of train_labels
print("Number of tags: {}".format(len(train_df.label.unique())))
frequencies = train_df.label.value_counts()
frequencies

Number of tags: 9


O                             385364
Disease                        20104
Chemical                       15582
DiseaseOrPhenotypicFeature      5762
GeneOrGeneProduct               5011
ChemicalEntity                  4633
OrganismTaxon                   1554
SequenceVariant                 1289
CellLine                         113
Name: label, dtype: int64

In [0]:
target_labels = ['Chemical', 'ChemicalEntity', 'O']
train_df = train_df[train_df['label'].isin(target_labels)]

In [0]:
train_df['label'] = train_df['label'].replace(['ChemicalEntity'], 'Chemical')

In [0]:
print("Number of tags: {}".format(len(train_df.label.unique())))
frequencies = train_df.label.value_counts()
frequencies

Number of tags: 2


O           385364
Chemical     20215
Name: label, dtype: int64

In [0]:
string_labels = []
for item in test_data:
  text = item['Text']
  id= item['id']
  sentences_split = nltk.word_tokenize(item["Text"])
  # print(sentences_split)
  try:
      token_spans = align_tokens(sentences_split,text)
  except:
      print("Error")
      # print(token_spans)
  for i in range(len(token_spans)):
      token_in_annotation = False
      for annotation in item['annotation']:
          if int(annotation["start"])<= token_spans[i][0] and int(annotation['end'])>=token_spans[i][1]:
              string_labels.append((text[token_spans[i][0]:token_spans[i][1]],annotation['type']))
              token_in_annotation = True
      if(token_in_annotation==False):
          string_labels.append((text[token_spans[i][0]:token_spans[i][1]],'O'))


Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


In [0]:
#addding data to dataframe
test_df = pd.DataFrame(string_labels, columns =['token', 'label'])


In [0]:
print("Number of tags: {}".format(len(test_df.label.unique())))
frequencies = test_df.label.value_counts()
frequencies

Number of tags: 3


O           290001
Disease      20104
Chemical     15582
Name: label, dtype: int64

In [0]:
target_labels = ['Chemical', 'O']
test_df = test_df[test_df['label'].isin(target_labels)]

In [0]:
# test_df['label'] = test_df['label'].replace(['ChemicalEntity'], 'Chemical')

In [0]:
test_df['label'].unique()

array(['O', 'Chemical'], dtype=object)

In [0]:
print("Number of tags: {}".format(len(test_df.label.unique())))
frequencies = test_df.label.value_counts()
frequencies

Number of tags: 2


O           290001
Chemical     15582
Name: label, dtype: int64

In [0]:
test_df

,token,label
3,and,O
4,lithium,Chemical
5,carbonate,Chemical
7,in,O
8,a,O
...,...,...
325682,challenge,O
325683,tests,O
325684,in,O
325685,rodents,O


In [0]:
train_df['group'] = (train_df['token'] == '.').cumsum()


In [0]:
train_n_df = train_df.groupby('group').agg({'token': ' '.join, 'label': ' '.join}).reset_index(drop=True)

train_n_df.rename(columns = {'token':'text', 'label':'labels'}, inplace = True)

In [0]:
# Split labels based on whitespace and turn them into a list
labels = [i.split() for i in train_n_df['labels'].values.tolist()]

# Check how many labels are there in the dataset
unique_labels = set()

for lb in labels:
  [unique_labels.add(i) for i in lb if i not in unique_labels]

In [0]:
# Map each label into its id representation and vice versa
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}
print(labels_to_ids)

{'Chemical': 0, 'O': 1}


In [0]:
train_n_df

,text,labels
0,due to a new deletion in the protein,O O O O O O O O
1,. OBJECTIVE : ( ) is a rare disorder character...,O O O O O O O O O O O O O O O Chemical O O O O...
2,. When present the defect is also found in the...,O O O O O O O O O O O O O O O O O O O O O O O ...
3,. : We describe a with identified at the 3rd m...,O O O O O O O O O O O O O
4,". The diagnosis of was suspected because of , ...",O O O O O O O O O O O O O Chemical O O O O O O O
...,...,...
17949,". By contrast , apomorphine-induced locomotion...",O O O O O O O O O O O O O O
17950,. Dopamine turnover ratios ( DOPAC : DA and HV...,O Chemical O O O Chemical O Chemical O Chemica...
17951,". However , apomorphine-induced reductions in ...",O O O O O O O Chemical O O O O O O O O O O
17952,. The implications of these findings are discu...,O O O O O O O O O O O O O O O O O O


In [0]:
test_df['group'] = (test_df['token'] == '.').cumsum()
test_n_df = test_df.groupby('group').agg({'token': ' '.join, 'label': ' '.join}).reset_index(drop=True)

test_n_df.rename(columns = {'token':'text', 'label':'labels'}, inplace = True)

In [0]:
test_n_df

,text,labels
0,and lithium carbonate in a newborn infant,O Chemical Chemical O O O O
1,". A newborn with massive , , , and a high seru...",O O O O O O O O O O O O Chemical O O O
2,. This is the first patient to initially manif...,O O O O O O O O O O O O O O O O O O O O O Chem...
3,. Sixty-three percent of these infants had tri...,O O O O O O O O O O
4,. Lithium carbonate may be a factor in the inc...,O Chemical Chemical O O O O O O O O O O O O O O
...,...,...
13599,". By contrast , apomorphine-induced locomotion...",O O O O O O O O O O O O O O
13600,. Dopamine turnover ratios ( DOPAC : DA and HV...,O Chemical O O O Chemical O Chemical O Chemica...
13601,". However , apomorphine-induced reductions in ...",O O O O O O O Chemical O O O O O O O O O O
13602,. The implications of these findings are discu...,O O O O O O O O O O O O O O O O O O


In [0]:
# # Split labels based on whitespace and turn them into a list
# labels = [i.split() for i in new_df['labels'].values.tolist()]

# # Check how many labels are there in the dataset
# unique_labels = set()

# for lb in labels:
#   [unique_labels.add(i) for i in lb if i not in unique_labels]

In [0]:

from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

In [0]:
train_txt=train_n_df['text'].values.tolist()
train_text_tokenized = tokenizer(train_txt, padding='max_length', max_length=512, truncation=True, return_tensors="pt")
train_input_ids=train_text_tokenized['input_ids']
train_attention_masks=train_text_tokenized['attention_mask']

In [0]:
test_txt=test_n_df['text'].values.tolist()
test_text_tokenized = tokenizer(test_txt, padding='max_length', max_length=512, truncation=True, return_tensors="pt")
test_input_ids=test_text_tokenized['input_ids']
test_attention_masks=test_text_tokenized['attention_mask']

In [0]:
import torch

def align_label(texts, labels):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)
    label_all_tokens=True

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]])
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids

In [0]:
train_lb = [i.split() for i in train_n_df['labels'].values.tolist()]

train_txt = train_n_df['text'].values.tolist()

train_new_label =[align_label(i,j) for i,j in zip(train_txt, train_lb)]

In [0]:
test_lb = [i.split() for i in test_n_df['labels'].values.tolist()]

test_txt = test_n_df['text'].values.tolist()

test_new_label =[align_label(i,j) for i,j in zip(test_txt, test_lb)]

In [0]:
train_pt_input_ids = torch.stack(list(train_input_ids), dim=0)

train_pt_attention_masks = torch.stack(list(train_attention_masks), dim=0)

train_pt_labels = torch.tensor(train_new_label, dtype=torch.long)

In [0]:
test_pt_input_ids = torch.stack(list(test_input_ids), dim=0)

test_pt_attention_masks = torch.stack(list(test_attention_masks), dim=0)

test_pt_labels = torch.tensor(test_new_label, dtype=torch.long)

In [0]:
from torch.utils.data import TensorDataset, random_split
import random
# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(train_pt_input_ids, train_pt_attention_masks, train_pt_labels)
test_dataset= TensorDataset(test_pt_input_ids, test_pt_attention_masks, test_pt_labels)


train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

test_size=len(test_dataset)
train_dataset, valid_dataset = random_split(train_dataset, [train_size, val_size])


print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
print('{:>5,} testing samples'.format(test_size))

14,363 training samples
3,591 validation samples
13,604 testing samples


In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


batch_size = 16

train_dataloader = DataLoader(train_dataset, sampler = RandomSampler(train_dataset), batch_size = batch_size )

validation_dataloader = DataLoader(valid_dataset, sampler = SequentialSampler(valid_dataset), batch_size = batch_size)

test_dataloader=DataLoader(test_dataset, sampler = SequentialSampler(test_dataset), batch_size = batch_size )


In [0]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [0]:

from transformers import BertForTokenClassification, AdamW, BertConfig
model = BertForTokenClassification.from_pretrained('bert-base-cased', num_labels=len(labels_to_ids))
model.cuda()

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cas

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [0]:
# defining the optimizer

epochs = 3
lr=2e-05
optimizer = AdamW(model.parameters(),
                  lr # args.learning_rate - default is 5e-5, our notebook had 2e-5
                   # args.adam_epsilon  - default is 1e-8.
                )

/local_disk0/.ephemeral_nfs/cluster_libraries/python/lib/python3.10/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [0]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [0]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [0]:
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import precision_score, recall_score, f1_score

In [0]:
import mlflow

In [0]:

mlflow.start_run()
loss_values = []
val_loss_vaues=[]
total_t0 = time.time()
for epoch in range(epochs):
  print('======== Epoch {:} / {:} ========'.format(epoch + 1, epochs))
  print('Training...')
  # Measure how long the training epoch takes.
  training_stats = []
  model.train()
  total_loss = 0
  t0 = time.time()
  for step, batch in enumerate(train_dataloader):
    if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)

    optimizer.zero_grad()
    loss, logits = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask, 
                            labels=b_labels).to_tuple()
    total_loss += loss.item()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    scheduler.step()

  avg_train_loss = total_loss / len(train_dataloader)
  loss_values.append(avg_train_loss)
  # Measure how long this epoch took.
  training_time = format_time(time.time() - t0)
  
  print("")
  print("  Average training loss: {0:.2f}".format(avg_train_loss))
  # print("  Average training loss: {0:.2f}".format(avg_train_loss))
  print("  Training epcoh took: {:}".format(training_time))
  # Validation
  print("Running Validation...")


  t0 = time.time()



  model.eval()
  total_eval_loss = 0
  val_predictions = []
  for batch in validation_dataloader:
    b_input_ids = batch[0].to(device)
    b_input_mask = batch[1].to(device)
    b_labels = batch[2].to(device)
    with torch.no_grad():
      loss, logits = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask,labels=b_labels).to_tuple()
    total_eval_loss += loss.item()
    predicted_labels = torch.argmax(logits, dim=2)
    val_predictions.extend(predicted_labels.detach().cpu().numpy())

  avg_val_loss = total_eval_loss / len(validation_dataloader)
  val_loss_vaues.append(avg_val_loss)
  validation_time = format_time(time.time() - t0)

  print("  Average validation loss: {0:.2f}".format(avg_val_loss))
  mlflow.log_param("lr", lr)
  mlflow.log_metric("epoch", epoch + 1)
  mlflow.log_metric("average_train_loss", avg_train_loss, step=epoch+1)
  mlflow.log_metric("average_val_loss", avg_val_loss,step=epoch+1)
  
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

mlflow.end_run()

======== Epoch 1 / 3 ========
Training...
  Batch    40  of    898.    Elapsed: 0:00:53.
  Batch    80  of    898.    Elapsed: 0:01:46.
  Batch   120  of    898.    Elapsed: 0:02:39.
  Batch   160  of    898.    Elapsed: 0:03:32.
  Batch   200  of    898.    Elapsed: 0:04:23.
  Batch   240  of    898.    Elapsed: 0:05:14.
  Batch   280  of    898.    Elapsed: 0:06:06.
  Batch   320  of    898.    Elapsed: 0:06:59.
  Batch   360  of    898.    Elapsed: 0:07:52.
  Batch   400  of    898.    Elapsed: 0:08:45.
  Batch   440  of    898.    Elapsed: 0:09:36.
  Batch   480  of    898.    Elapsed: 0:10:27.
  Batch   520  of    898.    Elapsed: 0:11:19.
  Batch   560  of    898.    Elapsed: 0:12:12.
  Batch   600  of    898.    Elapsed: 0:13:05.
  Batch   640  of    898.    Elapsed: 0:14:00.
  Batch   680  of    898.    Elapsed: 0:14:54.
  Batch   720  of    898.    Elapsed: 0:15:49.
  Batch   760  of    898.    Elapsed: 0:16:44.
  Batch   800  of    898.    Elapsed: 0:17:38.
  Batch   840  of 

In [0]:
# print('Predicting labels for {:,} test sentences...'.format(len(pt_input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in test_dataloader:
  # Add batch to GPU
  b_input_ids = batch[0].to(device)
  b_input_mask = batch[1].to(device)
  b_labels = batch[2].to(device)
  
  # Telling the model not to compute or store gradients, saving memory and 

  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

    DONE.


In [0]:
from sklearn.metrics import f1_score

# First, combine the results across the batches.
all_predictions = np.concatenate(predictions, axis=0)
all_true_labels = np.concatenate(true_labels, axis=0)

print("After flattening the batches, the predictions have shape:")
print("    ", all_predictions.shape)

# Next, let's remove the third dimension (axis 2), which has the scores
# for all 18 labels. 

# For each token, pick the label with the highest score.
predicted_label_ids = np.argmax(all_predictions, axis=2)

print("\nAfter choosing the highest scoring label for each token:")
print("    ", predicted_label_ids.shape) 


# Eliminate axis 0, which corresponds to the sentences.
predicted_label_ids = np.concatenate(predicted_label_ids, axis=0)
all_true_labels = np.concatenate(all_true_labels, axis=0)

print("\nAfter flattening the sentences, we have predictions:")
print("    ", predicted_label_ids.shape)
print("and ground truth:")
print("    ", all_true_labels.shape)

After flattening the batches, the predictions have shape:
     (13604, 512, 2)

After choosing the highest scoring label for each token:
     (13604, 512)

After flattening the sentences, we have predictions:
     (6965248,)
and ground truth:
     (6965248,)


In [0]:
# Construct new lists of predictions which don't include any null tokens.
real_token_predictions = []
real_token_labels = []

# For each of the input tokens in the dataset...
for i in range(len(all_true_labels)):

    # If it's not a token with a null label...
    if not all_true_labels[i] == -100:
        
        # Add the prediction and the ground truth to their lists.
        real_token_predictions.append(predicted_label_ids[i])
        real_token_labels.append(all_true_labels[i])

print("Before filtering out `null` tokens, length = {:,}".format(len(all_true_labels)))
print(" After filtering out `null` tokens, length = {:,}".format(len(real_token_labels)))

Before filtering out `null` tokens, length = 6,965,248
 After filtering out `null` tokens, length = 405,261


In [0]:
from sklearn.metrics import f1_score
# from seqeval.metrics import classification_report


# f1 = f1_score(real_token_labels, real_token_predictions, average='micro')
labels = [ids_to_labels[id.item()] for id in real_token_labels]
predictions = [ids_to_labels[id.item()] for id in real_token_predictions]

# print ("F1 score: {:.2%}".format(f1))

In [0]:
# from seqeval.metrics import classification_report
from sklearn.metrics import classification_report

print(classification_report(labels, predictions))

              precision    recall  f1-score   support

    Chemical       0.95      0.91      0.93     41066
           O       0.99      0.99      0.99    364195

    accuracy                           0.99    405261
   macro avg       0.97      0.95      0.96    405261
weighted avg       0.99      0.99      0.99    405261



In [0]:
report = classification_report(labels, predictions, output_dict=True)

df = pd.DataFrame(report).transpose()

In [0]:
df.to_csv("/dbfs/FileStore/Chanchal/logs/bio,cdr---cdr2.csv", index = True)

In [0]:
# import pandas as pd
# df1=pd.read_csv("/dbfs/FileStore/Chanchal/logs/bio,cdr---cdr.csv")

In [0]:
# df1

In [0]:
jklk

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-1796807091310185>:1
----> 1 jklk

NameError: name 'jklk' is not defined

In [0]:
with open("/dbfs/FileStore/Chanchal/Data_json/biored.json", "r") as f:
  test_data = json.load(f)

In [0]:
string_labels = []
for item in test_data:
  text = item['Text']
  id= item['id']
  sentences_split = nltk.word_tokenize(item["Text"])
  # print(sentences_split)
  try:
      token_spans = align_tokens(sentences_split,text)
  except:
      print("Error")
      # print(token_spans)
  for i in range(len(token_spans)):
      token_in_annotation = False
      for annotation in item['annotation']:
          if int(annotation["start"])<= token_spans[i][0] and int(annotation['end'])>=token_spans[i][1]:
              string_labels.append((text[token_spans[i][0]:token_spans[i][1]],annotation['type']))
              token_in_annotation = True
      if(token_in_annotation==False):
          string_labels.append((text[token_spans[i][0]:token_spans[i][1]],'O'))

Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error
Error


In [0]:
#addding data to dataframe
test_df = pd.DataFrame(string_labels, columns =['token', 'label'])

In [0]:
print("Number of tags: {}".format(len(test_df.label.unique())))
frequencies = test_df.label.value_counts()
frequencies

Number of tags: 7


O                             141466
DiseaseOrPhenotypicFeature      8963
GeneOrGeneProduct               8168
ChemicalEntity                  4634
SequenceVariant                 2738
OrganismTaxon                   2195
CellLine                         164
Name: label, dtype: int64

In [0]:
target_labels = ['ChemicalEntity', 'O']
test_df = test_df[test_df['label'].isin(target_labels)]

In [0]:
test_df['label'] = test_df['label'].replace(['ChemicalEntity'], 'Chemical')

In [0]:
test_df['label'].unique()

array(['O', 'Chemical'], dtype=object)

In [0]:
print("Number of tags: {}".format(len(test_df.label.unique())))
frequencies = test_df.label.value_counts()
frequencies

Number of tags: 2


O           141466
Chemical      4634
Name: label, dtype: int64

In [0]:
test_df['group'] = (test_df['token'] == '.').cumsum()
test_n_df = test_df.groupby('group').agg({'token': ' '.join, 'label': ' '.join}).reset_index(drop=True)

test_n_df.rename(columns = {'token':'text', 'label':'labels'}, inplace = True)

In [0]:
test_txt=test_n_df['text'].values.tolist()
test_text_tokenized = tokenizer(test_txt, padding='max_length', max_length=512, truncation=True, return_tensors="pt")
test_input_ids=test_text_tokenized['input_ids']
test_attention_masks=test_text_tokenized['attention_mask']

In [0]:
import torch

def align_label(texts, labels):
    tokenized_inputs = tokenizer(texts, padding='max_length', max_length=512, truncation=True)
    label_all_tokens=True

    word_ids = tokenized_inputs.word_ids()

    previous_word_idx = None
    label_ids = []

    for word_idx in word_ids:

        if word_idx is None:
            label_ids.append(-100)

        elif word_idx != previous_word_idx:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]])
            except:
                label_ids.append(-100)
        else:
            try:
                label_ids.append(labels_to_ids[labels[word_idx]] if label_all_tokens else -100)
            except:
                label_ids.append(-100)
        previous_word_idx = word_idx

    return label_ids

In [0]:
test_lb = [i.split() for i in test_n_df['labels'].values.tolist()]

test_txt = test_n_df['text'].values.tolist()

test_new_label =[align_label(i,j) for i,j in zip(test_txt, test_lb)]

In [0]:
test_pt_input_ids = torch.stack(list(test_input_ids), dim=0)

test_pt_attention_masks = torch.stack(list(test_attention_masks), dim=0)

test_pt_labels = torch.tensor(test_new_label, dtype=torch.long)

In [0]:
test_dataset= TensorDataset(test_pt_input_ids, test_pt_attention_masks, test_pt_labels)

In [0]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

test_dataloader=DataLoader(test_dataset, sampler = SequentialSampler(test_dataset), batch_size = batch_size )

In [0]:
# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in test_dataloader:
  # Add batch to GPU
  b_input_ids = batch[0].to(device)
  b_input_mask = batch[1].to(device)
  b_labels = batch[2].to(device)
  
  # Telling the model not to compute or store gradients, saving memory and 

  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                      attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

    DONE.


In [0]:
from sklearn.metrics import f1_score

# First, combine the results across the batches.
all_predictions = np.concatenate(predictions, axis=0)
all_true_labels = np.concatenate(true_labels, axis=0)

print("After flattening the batches, the predictions have shape:")
print("    ", all_predictions.shape)

# Next, let's remove the third dimension (axis 2), which has the scores
# for all 18 labels. 

# For each token, pick the label with the highest score.
predicted_label_ids = np.argmax(all_predictions, axis=2)

print("\nAfter choosing the highest scoring label for each token:")
print("    ", predicted_label_ids.shape) 


# Eliminate axis 0, which corresponds to the sentences.
predicted_label_ids = np.concatenate(predicted_label_ids, axis=0)
all_true_labels = np.concatenate(all_true_labels, axis=0)

print("\nAfter flattening the sentences, we have predictions:")
print("    ", predicted_label_ids.shape)
print("and ground truth:")
print("    ", all_true_labels.shape)

After flattening the batches, the predictions have shape:
     (6369, 512, 2)

After choosing the highest scoring label for each token:
     (6369, 512)

After flattening the sentences, we have predictions:
     (3260928,)
and ground truth:
     (3260928,)


In [0]:
# Construct new lists of predictions which don't include any null tokens.
real_token_predictions = []
real_token_labels = []

# For each of the input tokens in the dataset...
for i in range(len(all_true_labels)):

    # If it's not a token with a null label...
    if not all_true_labels[i] == -100:
        
        # Add the prediction and the ground truth to their lists.
        real_token_predictions.append(predicted_label_ids[i])
        real_token_labels.append(all_true_labels[i])

print("Before filtering out `null` tokens, length = {:,}".format(len(all_true_labels)))
print(" After filtering out `null` tokens, length = {:,}".format(len(real_token_labels)))

Before filtering out `null` tokens, length = 3,260,928
 After filtering out `null` tokens, length = 189,871


In [0]:
labels = [ids_to_labels[id.item()] for id in real_token_labels]
predictions = [ids_to_labels[id.item()] for id in real_token_predictions]

In [0]:
from sklearn.metrics import classification_report

print(classification_report(labels, predictions))

              precision    recall  f1-score   support

    Chemical       0.95      0.85      0.90     10998
           O       0.99      1.00      0.99    178873

    accuracy                           0.99    189871
   macro avg       0.97      0.92      0.94    189871
weighted avg       0.99      0.99      0.99    189871



In [0]:
report = classification_report(labels, predictions, output_dict=True)

df1 = pd.DataFrame(report).transpose()

In [0]:
df1.to_csv("/dbfs/FileStore/Chanchal/logs/bio+cdr__bio(chemical).csv", index = True)